In [2]:
!pip install tensorflow 

Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip show tensorflow

Name: tensorflow
Version: 2.11.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\modx\appdata\roaming\python\python39\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:

pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas matplotlib sklearn

Defaulting to user installation because normal site-packages is not writeable
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2936 sha256=2007ac5d012abde3a4636fbac77ff9f5c591260ff8db877c447c467f38cd9636
  Stored in directory: c:\users\modx\appdata\local\pip\cache\wheels\f8\e0\3d\9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os #for file paths
import pandas as pd
import tensorflow as tf
import numpy as np

### Import Data

In [7]:
#get the train and test csv file 
df = pd.read_csv("jigsaw-toxic-comment-classification-challenge/train.csv/train.csv")

In [8]:
df = pd.read_csv('jigsaw-toxic-comment-classification-challenge/train.csv/train.csv')


### Preprocessing

In [9]:
#textvector makes tokanization easy
from tensorflow.keras.layers import TextVectorization

In [10]:
#split the data into commnets and features

#for comments
x= df['comment_text']

#for other lables
y= df[df.columns[2:]].values

In [11]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [12]:
MAX_FEATURES = 200000 # number of words in the vocab


In [13]:
#set sentence length for deep learning model and max tokens, int is for words to integers

vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [14]:
#pass the numpy array (x.values) adapt to the comments
vectorizer.adapt(x.values)

In [15]:
vectorizer("hello world") #gives the numeric conversion

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([286, 261,   0, ...,   0,   0,   0], dtype=int64)>

In [16]:
#store in new dataset
vectorized_text = vectorizer(x.values)


In [17]:
#shape=(159571, 1800) 1st is for the sencences and 1800 is been capped. if sentence dosen't fit the max, pad out with zeros

vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32139,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [18]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices
#for data pipeline

#passing vectorized text and y-variables
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))

#chaches data
dataset = dataset.cache()
dataset = dataset.shuffle(160000)

#batch of 16
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [19]:
batch_x , batch_y = dataset.as_numpy_iterator().next()

In [20]:
batch_x.shape #16 samples max out at 1800 words each

(16, 1800)

In [21]:
batch_y.shape

(16, 6)

In [22]:
len(dataset) #9974 batches

9974

In [23]:
#70% for training
train = dataset.take(int(len(dataset)*.7))

#skip 70% and take 20% - for the validation
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))

#skip 90% take last 10% -- for the test
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

### Create Sequential Model

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [25]:
#initiate the model
model = Sequential()

# Create the embedding layer

#number of words and how many value long they are
model.add(Embedding(MAX_FEATURES+1, 32))

# Bidirectional LSTM Layer - Bidirectional RNN (backwards and forward)
#gup acceleration needs to be tanh
model.add(Bidirectional(LSTM(32, activation='tanh')))

# Feature extractor Fully connected layers - hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# Final layer #6 for the final outputs #0 and 1
model.add(Dense(6, activation='sigmoid'))

In [26]:
#binary for 0 and 1 o/p
model.compile(loss='BinaryCrossentropy', optimizer='Adam')


In [27]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

In [34]:
#fit the data

history = model.fit(train, epochs=1, validation_data=val)


6981/6981 [==============================] - 846s 121ms/step - loss: 0.0607 - val_loss: 0.0456


In [38]:
#checking gpu #run with tensorflow env

from tensorflow.python.client import device_lib
import tensorflow

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:GPU:0']


In [39]:
tensorflow.test.is_built_with_cuda()

True

### Make Predictions


In [40]:
input_text = vectorizer('You freaking suck! I am going to hit you.')


In [41]:
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7112,  396, ...,    0,    0,    0], dtype=int64)>

In [42]:
#test prediction
#model needs a np array imput shape

model.predict(np.array([input_text]))


1/1 [==============================] - 0s 133ms/step


array([[0.99726593, 0.17397907, 0.9490055 , 0.01346681, 0.76578814,
        0.08301122]], dtype=float32)

In [43]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [44]:
batch_x, batch_y = test.as_numpy_iterator().next()


In [45]:
(model.predict(batch_x) > 0.5).astype(int)


1/1 [==============================] - 0s 158ms/step


array([[1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

### Evaluate Model

In [46]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy


In [47]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [48]:
#loop through each batch

for batch in test.as_numpy_iterator(): 
    
    # Unpack the batch 
    X_true, y_true = batch
    
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions - converts in one big array
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    
    #update the state of metrics
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 65ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 62ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 65ms/step


1/1 [==============================] - 0s 61ms/step


In [50]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')


Precision: 0.8277922868728638, Recall:0.65503990650177, Accuracy:0.46339017152786255


### Test and Gradio

In [51]:
pip install gradio jinja2

     ------------------------------------ 139.5/139.5 kB 414.2 kB/s eta 0:00:00
  Using cached PyYAML-6.0-cp39-cp39-win_amd64.whl (151 kB)

     ---------------------------------------- 55.8/55.8 kB 2.8 MB/s eta 0:00:00


In [62]:
import tensorflow as tf
import gradio as gr

In [63]:
#saving the model
model.save('toxicity.h5')

In [64]:

#reload model
model = tf.keras.models.load_model('toxicity.h5')

In [65]:
#testing again
input_str = vectorizer('hey i freaken hate you!')


In [66]:
#set str to model
res = model.predict(np.expand_dims(input_str,0))


1/1 [==============================] - 0s 466ms/step


In [67]:
res


array([[0.8067019 , 0.01596382, 0.37168375, 0.01383506, 0.34510234,
        0.04634241]], dtype=float32)

In [68]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [69]:
#hook into gradio to show the results 


def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [70]:
#generate interface
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

C:\Users\modx\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\modx\AppData\Roaming\Python\Python39\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\modx\AppData\Roaming\Python\Python39\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [71]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://779afba3-2c56-4981.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 172ms/step


In [35]:
#last pin

#why use tf datapipeline
#for large dataset, load in batches to make it easier #it is in tf.data.dataset
#input pipeline benefits -- handle huge data sets by making batches and apply transformations to make data ready for training


#for tensorflow you have created new env - tensorflow use this as the kernal 
#enable tensorlfow-gpu and keras-gpu in this
#have added using the steps: https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084

